In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

abspath = os.path.abspath(os.getcwd())
finpath = Path(abspath).resolve().parent
parent_path = str(finpath) + '/data/'

train_path = parent_path + "train_set.csv"
df = pd.read_csv(train_path)

import warnings
warnings.filterwarnings("ignore")

Baseline model: Only gender taken into account

In [ ]:
correct = 0
observations = df.shape[0]
for x in range(observations):
    row = df.iloc[x]
    if row["Sex"] == 1:
        if row["Survived"] == 0:
            correct += 1
    elif row["Sex"] == 0:
        if row["Survived"] == 1:
            correct += 1
print("Accuracy:", correct / observations)

Accuracy: 78.9%

Basic logit model

In [67]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [54]:
df

,Survived,Age,Fare,Missing Data,1,2,3,C,Q,S,female,male
284,1,19.0,8.0500,1,0,0,1,0,0,1,0,1
502,0,21.0,7.7500,1,0,0,1,0,1,0,1,0
15,0,14.0,7.8542,1,0,0,1,0,0,1,1,0
767,0,NaN,39.6000,1,1,0,0,1,0,0,0,1
347,1,40.0,13.0000,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
507,1,33.0,26.0000,1,0,1,0,0,0,1,1,0
314,0,28.0,7.8958,1,0,0,1,0,0,1,0,1
768,0,30.5,7.7500,1,0,0,1,0,1,0,1,0
760,1,33.0,86.5000,0,1,0,0,0,0,1,1,0


In [68]:
df = df.dropna(axis=1)
y = df.iloc[:,0]
X = df.iloc[:,1:]

k = 5
kf = KFold(n_splits=k, random_state=None)

model = LogisticRegression(solver= 'liblinear')

acc_score = []

for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train , y_test = y.iloc[train_index], y.iloc[test_index]
     
    model.fit(X_train, y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))


accuracy of each fold - [0.7062937062937062, 0.8251748251748252, 0.7887323943661971, 0.7676056338028169, 0.8098591549295775]
Avg accuracy : 0.7795331429134247


In [69]:
val_path = parent_path + "validation_set.csv"
val_set = pd.read_csv(val_path)

In [70]:
val_set = val_set.dropna(axis=1)
X_val = val_set.iloc[:, 1:]
y_val = val_set.iloc[:, 0]

preds = model.predict(X_val)

acc = accuracy_score(preds, y_val)
print('Accuracy on validation set:', acc)

Accuracy on validation set: 0.8268156424581006


XGBoost

In [56]:
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [ ]:
y = df.iloc[:,0]
X = df.iloc[:,1:]


k = 5
kf = KFold(n_splits=k, random_state=None)

model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
acc_score = []

for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train , y_test = y.iloc[train_index], y.iloc[test_index]
     
    model.fit(X_train, y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)

avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

Accuracy: 81.3%

In [72]:
val_path = parent_path + "validation_set.csv"
val_set = pd.read_csv(val_path)

In [73]:
val_set = val_set.dropna(axis=1)
X_val = val_set.iloc[:, 1:]
y_val = val_set.iloc[:, 0]

preds = model.predict(X_val)

acc = accuracy_score(preds, y_val)
print('Accuracy on validation set:', acc)

Accuracy on validation set: 0.7932960893854749


: 